In [1]:
import pickle
with open('/root/nian/mollm_results/gcu/zgca,gpt-5-minimal/mols/gcu_conv2d_1010_42.pkl','rb') as f:
    a = pickle.load(f)
all_mols = a['all_mols']


In [2]:
len(all_mols)

123

In [3]:
all_mols[10][0].constraints


{'status': 'failed',
 'debug_log': "/root/kaiwu_test/workspace/topscc_kaiwu_pipeline/round2/depthwise_conv2d/33590/test.cc:223:39: error: no member named 'vfloat' in namespace 'tops'\n    const int V = tops::vlength<tops::vfloat>();\n                                ~~~~~~^\n1 error generated when compiling for gcu210.\n",
 'time_comparison': []}

In [27]:
print(a[1].value)


#include <tops/tops_runtime.h>
#include <tops.h>
#include <stdint.h>

using namespace tops;

static __host__ __forceinline__ int div_up_host(int a, int b) { return (a + b - 1) / b; }

// 改进点基于 parent#0：
// - 引入行间双缓冲+预取：对相邻输出行的输入3行集合做滚动复用（仅替换退出的那一行），减少DMA次数。
// - 对 dilation==1 的常见路径，构建“3行滑窗寄存器缓存”，在水平移动时只搬一组新列（以标量快速路径实现），减少随机访存判断。
// - 输出行仍整行写回；每线程专注一个通道集合。
// - 权重向量广播，常量化。

__global__ void dwconv3x3_row_rolling_prefetch_kernel(
    int C, int IH, int IW, int stride, int padding, int dilation,
    float* __restrict inp_g,
    float* __restrict weight_g,
    float* __restrict out_g)
{
    const int KH = 3, KW = 3;
    const int OH = (IH + 2 * padding - dilation * (KH - 1) - 1) / stride + 1;
    const int OW = (IW + 2 * padding - dilation * (KW - 1) - 1) / stride + 1;

    const int tid = threadIdx.x;
    const int tnum = blockDim.x;
    const int grid_threads = gridDim.x * tnum;

    const int V = tops::vlength<vfloat>(); // 32
    for (int c = blockIdx.x * tnum + tid; c < C; c += grid_t

In [37]:
a[2].constraints

{'status': 'success',
 'debug_log': 'testcase0 -> Passed\ntestcase1 -> Passed\ntestcase2 -> Passed\ntestcase3 -> Passed\ntestcase4 -> Passed\ntestcase5 -> Passed\ntestcase6 -> Passed\ntestcase7 -> Passed\ntestcase8 -> Passed\ntestcase9 -> Passed\n',
 'time_comparison': [{'name': 'GCU_CONV2D0',
   'avg': '24.85 us',
   'best': '20.06 us'},
  {'name': 'GCU_CONV2D1', 'avg': '86.22 us', 'best': '82.15 us'},
  {'name': 'GCU_CONV2D2', 'avg': '234.44 us', 'best': '195.95 us'},
  {'name': 'GCU_CONV2D3', 'avg': '2960.0 us', 'best': '2960.0 us'},
  {'name': 'GCU_CONV2D4', 'avg': '5390.0 us', 'best': '5390.0 us'},
  {'name': 'GCU_CONV2D5', 'avg': '315.81 us', 'best': '262.91 us'},
  {'name': 'GCU_CONV2D6', 'avg': '296.6 us', 'best': '227.96 us'},
  {'name': 'GCU_CONV2D7', 'avg': '2140.0 us', 'best': '1860.0 us'},
  {'name': 'GCU_CONV2D8', 'avg': '1050.0 us', 'best': '1050.0 us'},
  {'name': 'GCU_CONV2D9', 'avg': '43990.0 us', 'best': '43990.0 us'}]}

In [38]:
import re
scores = []
for item in a[2].constraints['time_comparison']:
    # 用正则提取数字部分
    avg = float(re.findall(r"[\d.]+", item['avg'])[0])
    best = float(re.findall(r"[\d.]+", item['best'])[0])
    score = best / avg
    scores.append(score)
    print(f"{item['name']}: best/avg = {score:.4f}")

mean_score = sum(scores) / len(scores)
print(f"\nOverall average score = {mean_score:.4f}")

GCU_CONV2D0: best/avg = 0.8072
GCU_CONV2D1: best/avg = 0.9528
GCU_CONV2D2: best/avg = 0.8358
GCU_CONV2D3: best/avg = 1.0000
GCU_CONV2D4: best/avg = 1.0000
GCU_CONV2D5: best/avg = 0.8325
GCU_CONV2D6: best/avg = 0.7686
GCU_CONV2D7: best/avg = 0.8692
GCU_CONV2D8: best/avg = 1.0000
GCU_CONV2D9: best/avg = 1.0000

Overall average score = 0.9066


In [17]:
import re
import pickle
root = '/root/nian/mollm_results/gcu/zgca,gpt-5-minimal/mols/gcu_rmsnorm_1010_42.pkl'
with open(root,'rb') as f:
    a = pickle.load(f)

all_mols = a['all_mols']

for mol, _ in reversed(all_mols):
    if mol.constraints['status'] == "success":
        # Step 2: 建立基准 best_time 字典
        print(mol.constraints)
        best_times = {}
        for d in mol.constraints["time_comparison"]:
            name = d["name"]
            best_us = float(re.findall(r"[\d.]+", d["best"])[0])
            best_times[name] = best_us
        break
print('best times:',best_times)

goal = all_mols[0][0].property_list[0]
print(goal)
for mol, _ in all_mols:
    con = mol.constraints

    # 没有 time_comparison → 0分
    if not con or "time_comparison" not in con:
        mol.total = 0.0
        mol.property[goal] = 100
        mol.scores = [1.0]
        continue

    # 把当前 item 的 time_comparison 转成字典形式方便查找
    item_times = {}
    for d in con["time_comparison"]:
        name = d.get("name")
        avg_str = d.get("avg")
        if name and avg_str:
            avg_us = float(re.findall(r"[\d.]+", avg_str)[0])
            item_times[name] = avg_us

    # 按 best_times 为主来循环
    scores = []
    for name, ref_best in best_times.items():
        if name not in item_times:
            # 这个 name 测试缺失（例如 MLP7）→ 记 0 分
            scores.append(0.0)
            continue
        
        avg_us = item_times[name]
        scores.append(ref_best / avg_us*100)
    mol.total = sum(scores) / len(scores) 
    mol.property[goal] = mol.total
    mol.scores = [1 - mol.total/100]

print("✅ 已根据最新基准重算完成。示例：")
kkk = sorted(all_mols,key=lambda x:x[0].total,reverse=True)
for i, (mol, _) in enumerate(kkk[:5]):  # 打印后5个看看
    print(f"{i}: total = {mol.total:.4f}")

{'status': 'success', 'debug_log': 'testcase0 -> Passed\ntestcase1 -> Passed\ntestcase2 -> Passed\ntestcase3 -> Passed\ntestcase4 -> Passed\ntestcase5 -> Passed\ntestcase6 -> Passed\ntestcase7 -> Passed\ntestcase8 -> Passed\ntestcase9 -> Passed\n', 'time_comparison': [{'name': 'GCU_RMSNORM0', 'avg': '27.13 us', 'best': '20.24 us'}, {'name': 'GCU_RMSNORM1', 'avg': '34.91 us', 'best': '21.83 us'}, {'name': 'GCU_RMSNORM2', 'avg': '40.96 us', 'best': '22.24 us'}, {'name': 'GCU_RMSNORM3', 'avg': '38.19 us', 'best': '22.43 us'}, {'name': 'GCU_RMSNORM4', 'avg': '97.52 us', 'best': '30.45 us'}, {'name': 'GCU_RMSNORM5', 'avg': '107.3 us', 'best': '37.5 us'}, {'name': 'GCU_RMSNORM6', 'avg': '186.38 us', 'best': '56.75 us'}, {'name': 'GCU_RMSNORM7', 'avg': '205.49 us', 'best': '63.09 us'}, {'name': 'GCU_RMSNORM8', 'avg': '496.09 us', 'best': '150.19 us'}, {'name': 'GCU_RMSNORM9', 'avg': '736.28 us', 'best': '280.75 us'}]}
best times: {'GCU_RMSNORM0': 20.24, 'GCU_RMSNORM1': 21.83, 'GCU_RMSNORM2': 

In [18]:
a['all_mols'] = all_mols
with open(root,'wb') as f:
    pickle.dump(a, f)

In [19]:
all_mols[kkk[0][1]-1][0].total

60.112287646200016

In [20]:
print(kkk[0][0].value)


#include <tops/tops_runtime.h>
#include <tops.h>

#define EPSILON 1e-6f

__device__ __forceinline__
void split_range_balanced(int n, int tid, int nth, int &beg, int &end) {
    int q = n / nth, r = n % nth;
    beg = tid * q + (tid < r ? tid : r);
    end = beg + q + (tid < r ? 1 : 0);
}

__attribute__((global, cooperative))
void rmsnorm_kernel(float* __restrict inp,
                    float* __restrict out,
                    int seq_len,
                    int dim) {
    if (seq_len <= 0 || dim <= 0) return;

    const int tid = threadIdx.x;
    const int nth = blockDim.x;

    int rb, re;
    split_range_balanced(seq_len, tid, nth, rb, re);
    if (rb >= re) return;

    tops_dte_ctx_t ctx_ld, ctx_st;
    ctx_ld.init();
    ctx_st.init();

    const int VL = tops::vlength<vfloat>(); // 32 elements
    const int KB64 = 65536, KB96 = 98304, KB128 = 131072;

    __valigned__ float rbuf0[KB128/sizeof(float)];
    __valigned__ float rbuf1[KB128/sizeof(float)];
    __valigned__ float 

In [ ]:
def fix_results(item):
    goal = item.property_list[0]
    print(goal)
    mol = item
    con = mol.constraints

    # 没有 time_comparison → 0分
    if not con or "time_comparison" not in con:
        mol.total = 0.0
        mol.property[goal] = 100
        mol.scores = [1.0]
        return mol

    # 把当前 item 的 time_comparison 转成字典形式方便查找
    item_times = {}
    for d in con["time_comparison"]:
        name = d.get("name")
        avg_str = d.get("avg")
        if name and avg_str:
            avg_us = float(re.findall(r"[\d.]+", avg_str)[0])
            item_times[name] = avg_us

    # 按 best_times 为主来循环
    scores = []
    for name, ref_best in best_times.items():
        if name not in item_times:
            # 这个 name 测试缺失（例如 MLP7）→ 记 0 分
            scores.append(0.0)
            continue
        
        avg_us = item_times[name]
        scores.append(ref_best / avg_us*100)
    mol.total = sum(scores) / len(scores) 
    mol.property[goal] = mol.total
    mol.scores = [1 - mol.total/100]
    return mol

In [171]:
a.keys()

dict_keys(['history', 'init_pops', 'final_pops', 'all_mols', 'properties', 'evaluation', 'running_time'])

In [ ]:
import json

data = {
    "title": "可落地的高性能经验",
    "author": "姜诗蕴",
    "content": "这是一段测试文字。"
}

# 正确写法：ensure_ascii=False
with open("example.json", "w", encoding="utf-8") as f:
    json.dump(data, f, ensure_ascii=False, indent=4)


## 石化分子

In [10]:
from rdkit import Chem, DataStructs
from rdkit.Chem import AllChem
import numpy as np
import random
from tdc import Oracle
diversity_scorer = Oracle(name='diversity')

def compute_tanimoto_matrix(fps):
    """计算指纹间的Tanimoto相似度矩阵"""
    n = len(fps)
    sim_matrix = np.zeros((n, n))
    for i in range(n):
        for j in range(i + 1, n):
            sim = DataStructs.TanimotoSimilarity(fps[i], fps[j])
            sim_matrix[i, j] = sim_matrix[j, i] = sim
    return sim_matrix

def maxmin_diversity_selection(smiles_list, subset_size=20, seed=42):
    """从SMILES列表中选出多样性最大的子集（MaxMin贪心法）"""
    random.seed(seed)
    mols = [Chem.MolFromSmiles(s) for s in smiles_list]
    mols = [i for i in mols if i is not None]
    smiles_list = [Chem.MolToSmiles(m) for m in mols]
    fps = [AllChem.GetMorganFingerprintAsBitVect(m, radius=2, nBits=2048) for m in mols]
    sim_matrix = compute_tanimoto_matrix(fps)
    
    n = len(smiles_list)
    # 随机选一个起点
    selected = [random.randint(0, n - 1)]
    remaining = list(set(range(n)) - set(selected))
    
    # 逐步选择与当前集合最不相似的分子
    while len(selected) < subset_size:
        # 计算每个未选分子与已选分子的最大相似度
        max_sims = [max(sim_matrix[i][selected]) for i in remaining]
        # 选择最大相似度最小的那个分子（即最不相似的）
        next_idx = remaining[int(np.argmin(max_sims))]
        selected.append(next_idx)
        remaining.remove(next_idx)
    
    # 计算最终子集的平均相似度
    sub_sims = []
    for i in range(len(selected)):
        for j in range(i + 1, len(selected)):
            sub_sims.append(sim_matrix[selected[i]][selected[j]])
    avg_sim = np.mean(sub_sims)
    diversity = 1 - avg_sim
    selected = [smiles_list[i] for i in selected]
    return selected, avg_sim, diversity

# ===============================
# 示例用法
# ===============================
# 假设你有50个分子（这里用简单示例代替）
smiles_list = [
    "CCO", "CCC", "CCN", "CCCl", "CCBr", "C1=CC=CC=C1", "C=CC", "CNC", "CC(C)O",
    "CCOC", "CCS", "CCF", "CC=O", "CCCO", "CCCN", "CCCF", "COC", "CCOCC", "C=CCCl",
    "CCNC(=O)O", "CCCCl", "CCCC", "CCBrO", "CCClO", "CC=CC", "CCCBr", "CCC(=O)O",
    "CC(C)Cl", "CC(C)Br", "CC(C)F", "CCCCO", "CCCCN", "CCCOC", "CCl", "CBr", "CF",
    "CCN(C)C", "CNC(=O)C", "CC(=O)N", "C(=O)O", "CC(O)C", "CC(C)C", "CCCOC(=O)N",
    "CC(O)CO", "CC(=O)O", "C=CO", "C=CCO", "C=CCN", "C=CCCl", "C=CCBr", "C=CCF"
]

best = None
for seed in range(10):
    sel, avg, div = maxmin_diversity_selection(smiles_list, 20, seed)
    if best is None or div > best[2]:
        best = (sel, avg, div)
print("最佳 Diversity:", best[2])
sel


最佳 Diversity: 0.8939171963589432


[21:30:31] DEPRECATION WARNING: please use MorganGenerator
[21:30:31] DEPRECATION WARNING: please use MorganGenerator
[21:30:31] DEPRECATION WARNING: please use MorganGenerator
[21:30:31] DEPRECATION WARNING: please use MorganGenerator
[21:30:31] DEPRECATION WARNING: please use MorganGenerator
[21:30:31] DEPRECATION WARNING: please use MorganGenerator
[21:30:31] DEPRECATION WARNING: please use MorganGenerator
[21:30:31] DEPRECATION WARNING: please use MorganGenerator
[21:30:31] DEPRECATION WARNING: please use MorganGenerator
[21:30:31] DEPRECATION WARNING: please use MorganGenerator
[21:30:31] DEPRECATION WARNING: please use MorganGenerator
[21:30:31] DEPRECATION WARNING: please use MorganGenerator
[21:30:31] DEPRECATION WARNING: please use MorganGenerator
[21:30:31] DEPRECATION WARNING: please use MorganGenerator
[21:30:31] DEPRECATION WARNING: please use MorganGenerator
[21:30:31] DEPRECATION WARNING: please use MorganGenerator
[21:30:31] DEPRECATION WARNING: please use MorganGenerat

['CCCCN',
 'c1ccccc1',
 'O=CO',
 'CC(C)Br',
 'C=CCCl',
 'CC=CC',
 'CNC',
 'CCNC(=O)O',
 'COC',
 'CCl',
 'CCN(C)C',
 'CF',
 'CC(N)=O',
 'CCOCC',
 'CC(O)CO',
 'CCCCl',
 'CCCOC',
 'CCBr',
 'CBr',
 'CCO']

In [ ]:
import pickle
with open('/root/nian/mollm_results/shihua_mol/zgca,gpt-5-minimal/mols/molecular_weight_sa_score_homo_lumo_gap_au_dipole_moment_debye_1020_43.pkl','rb') as f:
    a = pickle.load(f)
a.keys()

dict_keys(['history', 'init_pops', 'final_pops', 'all_mols', 'properties', 'evaluation', 'running_time', 'experience'])

In [4]:
mols = [i[0] for i in a['all_mols']]
mols[0].property


{'molecular_weight': 128.215,
 'sa_score': 3.0936156935295305,
 'homo_lumo_gap_au': 0.3147,
 'dipole_moment_debye': 1.4346}

In [5]:
import pandas as pd
df = pd.DataFrame({
    'SMILES': [i.value for i in mols],
    'molecular_weight': [i.property['molecular_weight'] for i in mols],
    'sa_score': [i.property['sa_score'] for i in mols],
    'homo_lumo_gap_au': [i.property['homo_lumo_gap_au'] for i in mols],
    'dipole_moment_debye': [i.property['dipole_moment_debye'] for i in mols]
})
df

,SMILES,molecular_weight,sa_score,homo_lumo_gap_au,dipole_moment_debye
0,CC(C)(C)C1CCCO1,128.215,3.093616,0.314700,1.434600
1,CC1CCC2CCC12O,126.199,3.793891,0.317100,1.258700
2,CC1OCC(O)CO1,118.132,3.278211,0.330300,1.901600
3,CC1CC2CCC2C1O,126.199,3.763289,0.316000,1.352800
4,COCCCC(O)CO,134.175,2.612727,0.315200,2.914600
...,...,...,...,...,...
147,CC1(C2COC2)COC1CO,158.197,4.120874,0.292758,1.617387
148,CC1OC(Cl)CC(O)C1Cl,185.050,4.916483,0.298186,0.670270
149,CC(=O)OC1CC(OC(C)(C)C)CO1,202.250,3.648001,0.263614,3.270246
150,CC(C)(C)OCCOC1COC(=O)C1,202.250,3.280035,0.259794,5.634473


In [8]:
constraints = [
    {"constraint_id": "test_001", "molecular_weight": [200, 350], "sa_score": [1, 4], "homo_lumo_gap": [0.25, 0.35], "dipole_moment": [1.0, 3.0]},
{"constraint_id": "test_002", "molecular_weight": [350, 500], "sa_score": [3, 6], "homo_lumo_gap": [0.20, 0.30], "dipole_moment": [2.0, 5.0]},
{"constraint_id": "test_003", "molecular_weight": [150, 250], "sa_score": [1, 3.5], "homo_lumo_gap": [0.30, 0.40], "dipole_moment": [0.0, 1.5]},
{"constraint_id": "test_004", "molecular_weight": [250, 400], "sa_score": [2.5, 5.5], "homo_lumo_gap": [0.22, 0.32], "dipole_moment": [4.0, 7.0]},
{"constraint_id": "test_005", "molecular_weight": [280, 320], "sa_score": [2.0, 3.5], "homo_lumo_gap": [0.29, 0.31], "dipole_moment": [1.5, 2.5]}
]
for constraint in constraints:
    df_filtered = df[
        (df["molecular_weight"].between(*constraint["molecular_weight"])) &
        (df["sa_score"].between(*constraint["sa_score"])) &
        (df["homo_lumo_gap_au"].between(*constraint["homo_lumo_gap"])) &
        (df["dipole_moment_debye"].between(*constraint["dipole_moment"]))
    ]
    print(df_filtered.shape)


(10, 5)
(0, 5)
(1, 5)
(0, 5)
(0, 5)
